Notebook to create the final list of subjects. Criteria:
- Have reactivity information.
- both stroop, msit and rest have full atlas coverage.
- be low motion

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import regex as re
import os 
from tqdm import tqdm
from nilearn import image, input_data
from joblib import Parallel, delayed

In [2]:
# subjects with reactivity
subjects_reactivity = pd.read_csv("../data/demo/pip_reactivity_2020.csv").id.to_list()
reactivity_df = pd.DataFrame({'SUB_ID': subjects_reactivity})

In [3]:
atlas_file = "../data/atlases/shen_2mm_268_parcellation.nii.gz"

def check_acquisition_task(img_file):
    """
    
    function to check that we have a full acquisition (280 scans) and 
    that atlas fit subject (all ROI have signal)
    
    """
    
    masker = input_data.NiftiLabelsMasker(atlas_file)
    roi_data = masker.fit_transform(img_file)
    n_obs = roi_data.shape[0]
    
    full_acq = False
    if n_obs == 280:
        full_acq = True
    
    roi_data_std = np.std(roi_data, axis=0)
    all_rois = ~np.any(roi_data_std == 0)
    is_ok = (full_acq==True) & (all_rois==True)
    
    return is_ok

def check_acquisition_rest(img_file):
    """
    
    function to check that we have a full acquisition (150 scans) and 
    and that atlas fit subject (all ROI have signal)
    
    """
    
    masker = input_data.NiftiLabelsMasker(atlas_file)
    roi_data = masker.fit_transform(img_file)
    n_obs = roi_data.shape[0]
    
    full_acq = False
    if n_obs == 150:
        full_acq = True
    
    roi_data_std = np.std(roi_data, axis=0)
    all_rois = ~np.any(roi_data_std == 0)
    
    is_ok = (full_acq==True) & (all_rois==True)
    
    return is_ok

In [4]:
task_id = "stroop"
pattern = "sub-(.*)_ses-01_task-%s_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz" % task_id
pattern_confounders = "../data/confounders/task-%s" % task_id + "/sub-%s" + "_ses-01_task-%s_desc-confounds_regressors.tsv" % task_id
list_subjects_ids = []
list_subjects_fwd = []

for filename in tqdm(os.listdir("../data/preproc_bold/task-%s" % task_id)):
    subj = re.findall(pattern=pattern, string=filename)[0]

    list_subjects_ids.append(int(subj))
    
    fwd = pd.read_csv(pattern_confounders % subj, sep="\t").framewise_displacement.dropna().mean()
    list_subjects_fwd.append(fwd)
    
stroop_df = pd.DataFrame({'SUB_ID': list_subjects_ids, 'FWD': list_subjects_fwd})
stroop_df.head()

parallel = Parallel(n_jobs=-1)
is_ok = parallel(delayed(check_acquisition_task)("../data/preproc_bold/task-%s" % task_id + "/" + filename) \
                     for filename in tqdm(os.listdir("../data/preproc_bold/task-%s" % task_id)))
is_ok = np.array(is_ok)

stroop_df = stroop_df.loc[is_ok,:]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 333/333 [03:40<00:00,  1.51it/s]


In [5]:
stroop_df.shape

(322, 2)

In [6]:
task_id = "msit"
pattern = "sub-(.*)_ses-01_task-%s_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz" % task_id
pattern_confounders = "../data/confounders/task-%s" % task_id + "/sub-%s" + "_ses-01_task-%s_desc-confounds_regressors.tsv" % task_id
list_subjects_ids = []
list_subjects_fwd = []

for filename in tqdm(os.listdir("../data/preproc_bold/task-%s" % task_id)):
    subj = re.findall(pattern=pattern, string=filename)[0]

    list_subjects_ids.append(int(subj))
    
    fwd = pd.read_csv(pattern_confounders % subj, sep="\t").framewise_displacement.dropna().mean()
    list_subjects_fwd.append(fwd)
    
msit_df = pd.DataFrame({'SUB_ID': list_subjects_ids, 'FWD': list_subjects_fwd})
msit_df.head()

parallel = Parallel(n_jobs=-1)
is_ok = parallel(delayed(check_acquisition_task)("../data/preproc_bold/task-%s" % task_id + "/" + filename) \
                     for filename in tqdm(os.listdir("../data/preproc_bold/task-%s" % task_id)))
is_ok = np.array(is_ok)

msit_df = msit_df.loc[is_ok,:]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 335/335 [03:32<00:00,  1.58it/s]


In [7]:
msit_df.shape

(317, 2)

In [8]:
task_id = "rest"
pattern = "sub-(.*)_ses-01_task-%s_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz" % task_id
pattern_confounders = "../data/confounders/task-%s" % task_id + "/sub-%s" + "_ses-01_task-%s_desc-confounds_regressors.tsv" % task_id
list_subjects_ids = []
list_subjects_fwd = []

for filename in tqdm(os.listdir("../data/preproc_bold/task-%s" % task_id)):
    subj = re.findall(pattern=pattern, string=filename)[0]

    list_subjects_ids.append(int(subj))
    
    fwd = pd.read_csv(pattern_confounders % subj, sep="\t").framewise_displacement.dropna().mean()
    list_subjects_fwd.append(fwd)
    
resting_df = pd.DataFrame({'SUB_ID': list_subjects_ids, 'FWD': list_subjects_fwd})
resting_df.head()

parallel = Parallel(n_jobs=-1)
is_ok = parallel(delayed(check_acquisition_rest)("../data/preproc_bold/task-%s" % task_id + "/" + filename) \
                     for filename in tqdm(os.listdir("../data/preproc_bold/task-%s" % task_id)))
is_ok = np.array(is_ok)

resting_df = resting_df.loc[is_ok,:]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 334/334 [01:52<00:00,  2.98it/s]


In [9]:
resting_df.shape

(309, 2)

Find intersection among subjects that have a low in-scanner motion (0.35 mm), measured using Power's method. We used this threshold as a compromise between number of subjects screened and motion

In [10]:
motion_thr = 0.35

stroop_low_motion_df = stroop_df[stroop_df.FWD < motion_thr]
msit_low_motion_df = msit_df[msit_df.FWD < motion_thr]
resting_low_motion_df = resting_df[resting_df.FWD < motion_thr]

subjects_intersect = list(set(reactivity_df.SUB_ID.to_list()) & \
                           set(stroop_low_motion_df.SUB_ID.to_list()) & \
                           set(msit_low_motion_df.SUB_ID.to_list()) & \
                           set(resting_low_motion_df.SUB_ID.to_list())
                          )

print(len(subjects_intersect))

np.savetxt("../data/subjects_intersect_motion_%s.txt" % str(motion_thr*100).split(".")[0].zfill(3),  
           subjects_intersect, fmt="%d")

242


In [11]:
# Age ranges
print(pd.merge(pd.DataFrame({'ePrime.id':np.loadtxt("../data/subjects_intersect_motion_035.txt")}),
         pd.read_spss("../data/demo/PIP_n330_03_26_2019.sav"), on = 'ePrime.id').age.min())
print(pd.merge(pd.DataFrame({'ePrime.id':np.loadtxt("../data/subjects_intersect_motion_035.txt")}),
         pd.read_spss("../data/demo/PIP_n330_03_26_2019.sav"), on = 'ePrime.id').age.max())

30.0
51.0


In [12]:
# Age mean and std
print(pd.merge(pd.DataFrame({'ePrime.id':np.loadtxt("../data/subjects_intersect_motion_035.txt")}),
         pd.read_spss("../data/demo/PIP_n330_03_26_2019.sav"), on = 'ePrime.id').age.mean())

print(pd.merge(pd.DataFrame({'ePrime.id':np.loadtxt("../data/subjects_intersect_motion_035.txt")}),
         pd.read_spss("../data/demo/PIP_n330_03_26_2019.sav"), on = 'ePrime.id').age.std())

40.00826446280992
6.226693797482864


In [13]:
# Sex counts
pd.merge(pd.DataFrame({'ePrime.id':np.loadtxt("../data/subjects_intersect_motion_035.txt")}),
         pd.read_spss("../data/demo/PIP_n330_03_26_2019.sav"), on = 'ePrime.id').gender.value_counts()

MALE      123
FEMALE    119
Name: gender, dtype: int64